In [142]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
import pandas as pd
import numpy as np
import time
from typing import List
import pickle

import sys

sys.path.insert(0, "../")  # needed for using the utils file in the notebook.
from utils import nutrition_facts_parser, sort_ingredients_alphabetically
from sentence_transformers import SentenceTransformer, util

from rich import inspect
from tqdm.notebook import tqdm

from transformers import AutoTokenizer, AutoModel
import torch

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

## merging the title of the recipes back:

In [144]:
recipes_path = "../data/full_dataset.csv"
recipe_title_link = pd.read_csv(
    recipes_path,
    index_col=0,
    usecols=["title", "link"],
    dtype={
        "title": "string[pyarrow]",
        "link": "string[pyarrow]",
    },
)
recipe_title_link = recipe_title_link.query(
    " link.str.contains('allrecipes.com') "
)


data = pd.read_csv("all_recipes_data_w_nutrition_facts_unstructured.csv")
data_w_nutrition = data.query(" Nutrition_facts_unstructured.notnull() ")
print(data_w_nutrition.shape)

data_w_nutrition.head()

(41080, 3)


,link,NER,Nutrition_facts_unstructured
0,www.allrecipes.com/recipe/241895/deconstructed...,"['orange juice', 'jiggers vodka']",Nutrition Facts\nServings Per Recipe 1\nCalori...
1,www.allrecipes.com/recipe/20808/kettle-corn/,"['vegetable oil', 'white sugar', 'popcorn kern...",Nutrition Facts\nServings Per Recipe 5\nCalori...
2,www.allrecipes.com/recipe/238733/pops-fabulous...,"['eggplants', 'green bell peppers', 'red bell ...",Nutrition Facts\nServings Per Recipe 10\nCalor...
3,www.allrecipes.com/recipe/241254/chipotle-mang...,"['avocados', 'tomatoes', 'mango', 'cilantro', ...",Nutrition Facts\nServings Per Recipe 20\nCalor...
4,www.allrecipes.com/recipe/261696/basic-homemad...,"['milk', 'heavy whipping cream', 'white vinega...",Nutrition Facts\nServings Per Recipe 4\nCalori...


In [145]:
data_w_nutrition = data_w_nutrition.merge(
    recipe_title_link.reset_index(),
    on="link",
    how="left",
    validate="1:1",
    copy=False,
)
print(data_w_nutrition.shape)
data_w_nutrition.head()

(41080, 4)


,link,NER,Nutrition_facts_unstructured,title
0,www.allrecipes.com/recipe/241895/deconstructed...,"['orange juice', 'jiggers vodka']",Nutrition Facts\nServings Per Recipe 1\nCalori...,Deconstructed Screwdriver (The Raw Egg)
1,www.allrecipes.com/recipe/20808/kettle-corn/,"['vegetable oil', 'white sugar', 'popcorn kern...",Nutrition Facts\nServings Per Recipe 5\nCalori...,Kettle Corn
2,www.allrecipes.com/recipe/238733/pops-fabulous...,"['eggplants', 'green bell peppers', 'red bell ...",Nutrition Facts\nServings Per Recipe 10\nCalor...,Pop'S Fabulous Ajvar
3,www.allrecipes.com/recipe/241254/chipotle-mang...,"['avocados', 'tomatoes', 'mango', 'cilantro', ...",Nutrition Facts\nServings Per Recipe 20\nCalor...,Chipotle-Mango Guacamole
4,www.allrecipes.com/recipe/261696/basic-homemad...,"['milk', 'heavy whipping cream', 'white vinega...",Nutrition Facts\nServings Per Recipe 4\nCalori...,Basic Homemade Ricotta Cheese


In [146]:
recipe_title_link.columns, data_w_nutrition.columns

(Index(['link'], dtype='object'),
 Index(['link', 'NER', 'Nutrition_facts_unstructured', 'title'], dtype='object'))

## sort NER ingredients alphabetically:

In [147]:
data_w_nutrition.NER = data_w_nutrition.NER.apply(eval)

data_w_nutrition["sorted_NER"] = data_w_nutrition.NER.apply(
    sort_ingredients_alphabetically
)
data_w_nutrition.head()

,link,NER,Nutrition_facts_unstructured,title,sorted_NER
0,www.allrecipes.com/recipe/241895/deconstructed...,"[orange juice, jiggers vodka]",Nutrition Facts\nServings Per Recipe 1\nCalori...,Deconstructed Screwdriver (The Raw Egg),"[jiggers vodka, orange juice]"
1,www.allrecipes.com/recipe/20808/kettle-corn/,"[vegetable oil, white sugar, popcorn kernels]",Nutrition Facts\nServings Per Recipe 5\nCalori...,Kettle Corn,"[popcorn kernels, vegetable oil, white sugar]"
2,www.allrecipes.com/recipe/238733/pops-fabulous...,"[eggplants, green bell peppers, red bell peppe...",Nutrition Facts\nServings Per Recipe 10\nCalor...,Pop'S Fabulous Ajvar,"[cooking spray, corn oil, eggplants, garlic, g..."
3,www.allrecipes.com/recipe/241254/chipotle-mang...,"[avocados, tomatoes, mango, cilantro, red onio...",Nutrition Facts\nServings Per Recipe 20\nCalor...,Chipotle-Mango Guacamole,"[avocados, black pepper, cayenne pepper, cilan..."
4,www.allrecipes.com/recipe/261696/basic-homemad...,"[milk, heavy whipping cream, white vinegar, salt]",Nutrition Facts\nServings Per Recipe 4\nCalori...,Basic Homemade Ricotta Cheese,"[heavy whipping cream, milk, salt, white vinegar]"


## Prepare for Sentence Transformer:
- create singular "recipes" from each unique ingredient in the corpus --> each ingredient would have its own embedding as well.
- get an embedding for each recipe based on the sorted NER ingredients.

In [148]:
all_ingredients = set(data_w_nutrition.NER.explode())
all_ingredients.remove(np.nan)
len(all_ingredients)

9900

In [149]:
data_w_nutrition.columns

Index(['link', 'NER', 'Nutrition_facts_unstructured', 'title', 'sorted_NER'], dtype='object')

In [150]:
# create a 1 ingredient recipe so that we can get the embedding of each ingredient:

singular_ingredient_recipes_data = []

for ingredient in all_ingredients:
    singular_ingredient_recipes_data.append(
        {
            "link": np.nan,
            "NER": [ingredient],
            "Nutrition_facts_unstructured": np.nan,
            "title": ingredient,
            "sorted_NER": [ingredient],
        }
    )
singular_ingredient_recipes = pd.DataFrame(
    data=singular_ingredient_recipes_data
)
print(singular_ingredient_recipes.shape)
singular_ingredient_recipes.head()

(9900, 5)


,link,NER,Nutrition_facts_unstructured,title,sorted_NER
0,NaN,[strawberry flavored],NaN,strawberry flavored,[strawberry flavored]
1,NaN,[graham crumbs],NaN,graham crumbs,[graham crumbs]
2,NaN,[Falafel],NaN,Falafel,[Falafel]
3,NaN,[serrano chiles],NaN,serrano chiles,[serrano chiles]
4,NaN,[pomelo],NaN,pomelo,[pomelo]


In [151]:
data_w_nutrition = pd.concat([data_w_nutrition, singular_ingredient_recipes])
print(data_w_nutrition.shape)

(50980, 5)


In [152]:
data_w_nutrition.isnull().sum()

link                            9900
NER                                0
Nutrition_facts_unstructured    9900
title                              0
sorted_NER                         0
dtype: int64

In [153]:
data_w_nutrition['sorted_joined_NER'] = data_w_nutrition.sorted_NER.str.join(" ")

data_w_nutrition.sample()

,link,NER,Nutrition_facts_unstructured,title,sorted_NER,sorted_joined_NER
8873,www.allrecipes.com/recipe/7994/coconut-poke-cake/,"[white cake, cream of coconut, condensed milk,...",Nutrition Facts\nServings Per Recipe 24\nCalor...,Coconut Poke Cake,"[coconut, condensed milk, cream of coconut, fr...",coconut condensed milk cream of coconut frozen...


In [154]:
# max ingredient count in a recipe:
max_number_ingredients_per_recipe = data_w_nutrition.NER.apply(len).max()
max_number_ingredients_per_recipe

37

In [155]:
device = "mps" if torch.backends.mps.is_available() else "cpu"


# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[
        0
    ]  # First element of model_output contains all token embeddings
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def get_embedding(sentence: str):
    # Load AutoModel from huggingface model repository
    tokenizer = AutoTokenizer.from_pretrained(
        "sentence-transformers/all-MiniLM-L6-v2"
    )
    model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
    model.to(device=device)
    # Tokenize sentences
    encoded_input = tokenizer(
        sentence,
        padding=True,
        truncation=True,
        max_length=max_number_ingredients_per_recipe,
        return_tensors="pt",
    ).to(device=device)

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, mean pooling
    sentence_embeddings = mean_pooling(
        model_output, encoded_input["attention_mask"]
    )
    return sentence_embeddings

# make sure we apply new indexing to retrieve the recipes later:
data_w_nutrition = data_w_nutrition.reset_index(drop=True)
# corpus:
# sentences = data_w_nutrition.sorted_joined_NER.tolist()


# sentences_embeddings = data_w_nutrition.sorted_joined_NER.progress_apply(get_embedding)

In [156]:
data_w_nutrition.sorted_joined_NER.values[0]

'jiggers vodka orange juice'

In [157]:
data_w_nutrition.head()

,link,NER,Nutrition_facts_unstructured,title,sorted_NER,sorted_joined_NER
0,www.allrecipes.com/recipe/241895/deconstructed...,"[orange juice, jiggers vodka]",Nutrition Facts\nServings Per Recipe 1\nCalori...,Deconstructed Screwdriver (The Raw Egg),"[jiggers vodka, orange juice]",jiggers vodka orange juice
1,www.allrecipes.com/recipe/20808/kettle-corn/,"[vegetable oil, white sugar, popcorn kernels]",Nutrition Facts\nServings Per Recipe 5\nCalori...,Kettle Corn,"[popcorn kernels, vegetable oil, white sugar]",popcorn kernels vegetable oil white sugar
2,www.allrecipes.com/recipe/238733/pops-fabulous...,"[eggplants, green bell peppers, red bell peppe...",Nutrition Facts\nServings Per Recipe 10\nCalor...,Pop'S Fabulous Ajvar,"[cooking spray, corn oil, eggplants, garlic, g...",cooking spray corn oil eggplants garlic green ...
3,www.allrecipes.com/recipe/241254/chipotle-mang...,"[avocados, tomatoes, mango, cilantro, red onio...",Nutrition Facts\nServings Per Recipe 20\nCalor...,Chipotle-Mango Guacamole,"[avocados, black pepper, cayenne pepper, cilan...",avocados black pepper cayenne pepper cilantro ...
4,www.allrecipes.com/recipe/261696/basic-homemad...,"[milk, heavy whipping cream, white vinegar, salt]",Nutrition Facts\nServings Per Recipe 4\nCalori...,Basic Homemade Ricotta Cheese,"[heavy whipping cream, milk, salt, white vinegar]",heavy whipping cream milk salt white vinegar


In [167]:

# recipe_title_embeddings = {}
with open('recipe_title_embeddings.pkl', "rb") as fIn:
    recipe_title_embeddings = pickle.load(fIn)


try:
    for index in tqdm(
        data_w_nutrition.index.values
    ):
        if str(index) not in recipe_title_embeddings.keys():
            title_index = str(index)
            print(title_index)
            ner = data_w_nutrition.iloc[index].sorted_joined_NER
            
            recipe_title_embeddings[title_index] = get_embedding(sentence=ner)
        else:
            print("in dict already !")
except (KeyboardInterrupt):
    with open('recipe_title_embeddings.pkl', "wb") as fOut:
        pickle.dump(recipe_title_embeddings, fOut, protocol=pickle.HIGHEST_PROTOCOL)


  0%|          | 0/50980 [00:00<?, ?it/s]

in dict already !
in dict already !
in dict already !
in dict already !
in dict already !
in dict already !
in dict already !
in dict already !
in dict already !
9
10
11
12
13
14
15
16


In [159]:
title_index

Index([0], dtype='int64')

In [ ]:
data_w_nutrition['embedding'] = sentences_embeddings
data_w_nutrition.to_csv('data_w_embeddings.csv', index=False)

In [ ]:
len(sentences_embeddings), len(sentences_embeddings[0]), inspect(sentences_embeddings)

In [ ]:
corpus_embeddings = corpus_embeddings.to(device)
corpus_embeddings = util.normalize_embeddings(corpus_embeddings)

query_embeddings = query_embeddings.to(device)
# Assuming query_embeddings is a list of individual embeddings
average_query_embedding = np.mean(query_embeddings, axis=0)
query_embeddings = util.normalize_embeddings(query_embeddings)

hits = util.semantic_search(
    query_embeddings, corpus_embeddings, score_function=util.dot_score
)

In [ ]:
inspect(get_embedding(sentence=ner))


╭──────────────────────────────────── <class 'torch.Tensor'> ────────────────────────────────────╮
│ ╭────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ tensor([[-9.8901e-01, -2.0382e-01, -4.9042e-01,  2.0556e-01, -1.1693e-01,                  │ │
│ │ │   │     1.9092e-02,  1.2386e+00,  1.1871e-01,  2.8387e-01, -6.4530e-01,                  │ │
│ │ │   │    -2.9526e-01,  1.6818e-01, -5.0491e-01,  2.5442e-01, -1.3514e-01,                  │ │
│ │ │   │    -7.4551e-03,  5.7074e-01, -5.0032e-02, -1.5800e-01, -4.4758e-01,                  │ │
│ │ │   │    -4.3686e-01, -8.2509e-01,  4.3340e-01, -3.1382e-01, -2.5056e-02,                  │ │
│ │ │   │     5.3372e-01, -1.3368e-02,  3.5514e-01,  2.4119e-01, -3.6043e-01,                  │ │
│ │ │   │     4.3452e-01, -1.1954e-01,  1.6866e-01, -4.4973e-01,  3.6119e-01,                  │ │
│ │ │   │    -6.6882e-01,  3.5813e-01, -1.3205e-01, -3.5581e-01,  2.5802e-01,                  │ │
│ │ │   │     4.6431e-01,  3.7824e-01,  2.5601e-02,  3.0820e-02,  2.9815e-01,                  │ │
│ │ │   │     1.6597e-02, -3.1659e-01,  2.9297e-01,  5.0394e-01, -1.9607e-01,                  │ │
│ │ │   │    -5.7422e-01, -3.8034e-01, -1.0344e-01, -2.5817e-01,  3.4692e-01,                  │ │
│ │ │   │    -2.9078e-01,  4.9361e-01,  6.1496e-02, -4.9098e-01,  8.3505e-01,                  │ │
│ │ │   │    -7.8031e-02,  1.7253e-01, -1.0245e-01,  1.2695e-01, -3.3273e-01,                  │ │
│ │ │   │    -5.9285e-01,  2.4383e-02,  6.8808e-01,  3.3818e-01,  3.8515e-01,                  │ │
│ │ │   │    -4.0990e-01,  2.1330e-01,  5.4807e-02,  8.3242e-01, -1.9225e-01,                  │ │
│ │ │   │    -2.8017e-01,  4.1307e-01,  2.5145e-01,  2.9946e-01,  2.0378e-01,                  │ │
│ │ │   │    -2.9498e-01, -3.8240e-01, -2.7366e-01,  2.1847e-01, -4.6744e-02,                  │ │
│ │ │   │     2.7236e-01, -3.4654e-01,  1.8434e-01, -3.8669e-02,  7.6205e-01,                  │ │
│ │ │   │    -2.8068e-01, -6.2159e-02,  3.9342e-01, -1.8654e-01,  2.1275e-02,                  │ │
│ │ │   │     3.4602e-02, -6.9712e-02,  5.0633e-02, -3.6245e-01,  6.0767e-01,                  │ │
│ │ │   │     7.9196e-02,  4.1972e-01,  3.4137e-01, -4.1307e-01,  1.7492e-01,                  │ │
│ │ │   │    -2.0730e-01, -1.4540e-01,  1.1275e-01,  4.6259e-01,  1.0088e-01,                  │ │
│ │ │   │    -1.1210e-01, -3.1101e-01,  1.8522e-01, -2.7323e-02, -2.6260e-01,                  │ │
│ │ │   │    -1.1856e-02,  3.2197e-01,  2.7004e-01, -3.2856e-01, -1.1007e-01,                  │ │
│ │ │   │     1.4835e-01,  2.1049e-01, -1.7083e-01,  3.7198e-01, -4.4367e-01,                  │ │
│ │ │   │    -1.3119e-02,  9.3158e-02,  2.8856e-33,  3.1962e-01, -3.6634e-01,                  │ │
│ │ │   │     1.4039e-01,  1.1813e-01,  1.3475e-01,  1.6443e-04, -1.5379e-01,                  │ │
│ │ │   │    -4.5311e-01,  9.6908e-02,  5.9725e-01, -5.7690e-03, -4.9543e-01,                  │ │
│ │ │   │    -5.8330e-01,  2.6431e-01,  4.4793e-01, -1.7828e-01, -2.4763e-01,                  │ │
│ │ │   │     2.3182e-02, -1.1146e-01, -7.3640e-02,  3.8166e-02, -6.7267e-02,                  │ │
│ │ │   │    -4.6709e-01,  3.4663e-01, -4.1355e-01,  2.5395e-01, -2.6704e-01,                  │ │
│ │ │   │    -1.6283e-01,  1.4912e-01,  3.3155e-01,  3.2113e-02,  3.8438e-03,                  │ │
│ │ │   │    -1.1464e-01, -8.8226e-02, -4.4411e-01, -3.4113e-01, -1.2218e-02,                  │ │
│ │ │   │    -4.0662e-01, -1.4964e-01,  1.8143e-01,  3.8000e-01, -2.2640e-01,                  │ │
│ │ │   │    -2.8149e-01,  6.8428e-01, -2.6941e-01, -5.7988e-01, -1.9347e-01,                  │ │
│ │ │   │     3.1378e-01,  5.8538e-01,  1.1732e-01, -4.9327e-01, -4.2642e-01,                  │ │
│ │ │   │     7.5409e-01,  2.5247e-01, -3.1609e-01,  2.1728e-01, -6.1165e-02,                  │ │
│ │ │   │     1.3293e-01,  7.0922e-02, -1.6198e-01, -2.1220e-01,  8.8215e-02,                  │ │
│ │ │   │    -3.8093e-01, -2.1061e-01, -